In [1]:
import sys
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.dynamicframe import DynamicFrame
from awsglue.job import Job
# from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType
from awsglue.transforms import *
from awsglue.gluetypes import *
import boto3
import json
import botocore
import os
import datetime
from typing import List, Dict

glueContext = GlueContext(SparkContext.getOrCreate())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Setup Test Parameters

In [2]:
sys.argv+= ["--data",'{"objectS3": "input/sap/test/FaltanteVirtual.txt", "source": "sap", "interface": "test", "file_name": "FaltanteVirtual.txt", "uuid_file": "7d0eb092e92c4cc19d3f6f9f01343110", "process_code": "20220709233628", "pt_year": "2022", "pt_month": "07", "pt_day": "09", "pt_hour": "23", "pt_minute": "36", "pt_second": "28", "pt_time": "233628", "bucket_raw": "auna-lake-raw-dev", "database_name_raw": "auna-lake-raw-dev", "bucket_artifacts": "auna-lake-artifacts", "key_artifacts":"schemas/sap/test/FaltanteVirtual.json", "bucket_landing": "auna-lake-landing", "key_landing_move": "move/sap/20220709233628/7d0eb092e92c4cc19d3f6f9f01343110/test_FaltanteVirtual.txt"}']
     

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Init Process

In [3]:
args = getResolvedOptions(sys.argv, ["data"])
args

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'job_bookmark_option': 'job-bookmark-disable', 'job_bookmark_from': None, 'job_bookmark_to': None, 'JOB_ID': None, 'JOB_RUN_ID': None, 'SECURITY_CONFIGURATION': None, 'encryption_type': None, 'enable_data_lineage': None, 'RedshiftTempDir': None, 'TempDir': None, 'data': '{"objectS3": "input/sap/test/FaltanteVirtual.txt", "source": "sap", "interface": "test", "file_name": "FaltanteVirtual.txt", "uuid_file": "7d0eb092e92c4cc19d3f6f9f01343110", "process_code": "20220709233628", "pt_year": "2022", "pt_month": "07", "pt_day": "09", "pt_hour": "23", "pt_minute": "36", "pt_second": "28", "pt_time": "233628", "bucket_raw": "auna-lake-raw-dev", "database_name_raw": "auna-lake-raw-dev", "bucket_artifacts": "auna-lake-artifacts", "key_artifacts":"schemas/sap/test/FaltanteVirtual.json", "bucket_landing": "auna-lake-landing", "key_landing_move": "move/sap/20220709233628/7d0eb092e92c4cc19d3f6f9f01343110/test_FaltanteVirtual.txt"}'}

In [4]:
data_interface = json.loads(args["data"])
data_interface

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'objectS3': 'input/sap/test/FaltanteVirtual.txt', 'source': 'sap', 'interface': 'test', 'file_name': 'FaltanteVirtual.txt', 'uuid_file': '7d0eb092e92c4cc19d3f6f9f01343110', 'process_code': '20220709233628', 'pt_year': '2022', 'pt_month': '07', 'pt_day': '09', 'pt_hour': '23', 'pt_minute': '36', 'pt_second': '28', 'pt_time': '233628', 'bucket_raw': 'auna-lake-raw-dev', 'database_name_raw': 'auna-lake-raw-dev', 'bucket_artifacts': 'auna-lake-artifacts', 'key_artifacts': 'schemas/sap/test/FaltanteVirtual.json', 'bucket_landing': 'auna-lake-landing', 'key_landing_move': 'move/sap/20220709233628/7d0eb092e92c4cc19d3f6f9f01343110/test_FaltanteVirtual.txt'}

In [6]:
bucket_raw = data_interface["bucket_raw"]
database_name_raw = data_interface["database_name_raw"]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
def s3_read_file(bucket, key):
    s3 = boto3.client('s3')
    file = s3.get_object(Bucket=bucket, Key=key)
    file = file['Body'].read()
    filebody = file.decode('utf-8', errors='replace')
    return filebody    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
bucket_artifacts = data_interface["bucket_artifacts"]
key_artifacts = data_interface["key_artifacts"]
data_schema = s3_read_file(bucket_artifacts, key_artifacts)
data_schema = json.loads(data_schema)
data_schema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'table_name': 'tbl_faltante_virtual', 'columns': [{'Name': 'centro', 'Type': 'string'}, {'Name': 'aniomes', 'Type': 'string'}, {'Name': 'fechaproceso', 'Type': 'string'}, {'Name': 'codcliente', 'Type': 'string'}, {'Name': 'codienda', 'Type': 'string'}, {'Name': 'cantidad', 'Type': 'string'}, {'Name': 'monto', 'Type': 'string'}, {'Name': 'deuda', 'Type': 'string'}, {'Name': 'tipo', 'Type': 'string'}], 'field.type': 'csv', 'field.delim': '\t', 'partitions_key': [{'Name': 'pt_year', 'Type': 'string'}, {'Name': 'pt_month', 'Type': 'string'}, {'Name': 'pt_day', 'Type': 'string'}, {'Name': 'pt_time', 'Type': 'string'}], 'primary_key': ['centro', 'aniomes', 'fechaproceso', 'codcliente', 'codtienda'], 'quotes': False}

In [10]:
def format_schema_type(column_type):
    if str.lower(column_type)== "string":
            return StringType()
    if str.lower(column_type)== "integer":
            return IntegerType()
    if str.lower(column_type)== "double":
            return DoubleType()
    return StringType()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def glue_spark_schema(data_schema):
    data_columns = data_schema["columns"]
    struct_type_lst = []
    
    for columns in data_columns:
        row_type = Field(columns["Name"],format_schema_type(columns["Type"]))
        struct_type_lst.append(row_type)
    return StructType(struct_type_lst)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
glue_schema = glue_spark_schema(data_schema)
glue_schema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

StructType([Field(centro, StringType({}), {}),Field(aniomes, StringType({}), {}),Field(fechaproceso, StringType({}), {}),Field(codcliente, StringType({}), {}),Field(codienda, StringType({}), {}),Field(cantidad, StringType({}), {}),Field(monto, StringType({}), {}),Field(deuda, StringType({}), {}),Field(tipo, StringType({}), {})], {})

In [13]:
bucket_landing = data_interface["bucket_landing"]
key_landing_move = data_interface["key_landing_move"]

connection_type = "s3"
connection_options = {"paths": [ f"s3://{bucket_landing}/{key_landing_move}"]}
format_options = {"withSchema": json.dumps(glue_schema.jsonValue()), "separator": data_schema["field.delim"]
        }

glue_raw = glueContext.create_dynamic_frame.from_options(
                connection_type = connection_type, 
                connection_options = connection_options, 
                format="csv", 
                format_options = format_options, 
                transformation_ctx="")

glue_raw.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- centro: string
|-- aniomes: string
|-- fechaproceso: string
|-- codcliente: string
|-- codienda: string
|-- cantidad: string
|-- monto: string
|-- deuda: string
|-- tipo: string

In [14]:
def append_partitions(glue_raw):
    
    def AddDateCol_year(r):  
        r["pt_year"] = data_interface["pt_year"]
        return r 

    def AddDateCol_month(r):  
        r["pt_month"] = data_interface["pt_month"]
        return r 

    def AddDateCol_day(r):  
        r["pt_day"] = data_interface["pt_day"]
        return r 

    def AddDateCol_time(r):  
        r["pt_time"] = data_interface["pt_time"]
        return r 
    
    partitions_key = [ partition["Name"] for partition in data_schema["partitions_key"]]
    
    if "pt_year" in partitions_key:
        glue_raw = Map.apply(frame = glue_raw, f = AddDateCol_year)
    if "pt_month" in partitions_key:
        glue_raw = Map.apply(frame = glue_raw, f = AddDateCol_month)
    if "pt_day" in partitions_key:
        glue_raw = Map.apply(frame = glue_raw, f = AddDateCol_day)
    if "pt_time" in partitions_key:
        glue_raw = Map.apply(frame = glue_raw, f = AddDateCol_time)
    
    return glue_raw

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
def append_audit(glue_raw):
    
    def AddDateCol_process_code(r):  
        r["process_code"] = data_interface["uuid_file"]
        return r 

    def AddDateCol_uuid(r):  
        r["uuid_file"] = data_interface["process_code"]
        return r
    
    audit_key = [{"Name":"process_code", "Type":"string"},
            {"Name":"uuid_file", "Type":"string"}]
    data_schema["columns"] = data_schema["columns"]+audit_key


    glue_raw = Map.apply(frame = glue_raw, f = AddDateCol_process_code)
    glue_raw = Map.apply(frame = glue_raw, f = AddDateCol_uuid)
    
    return glue_raw

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
def data_sort_columns(glue_raw):
    column_total = [col["Name"] for col in data_schema["columns"]]
    partitions_key = [col["Name"] for col in data_schema["partitions_key"]] 
    column_total+= partitions_key
    return glue_raw.select_fields(column_total)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
glue_raw.toDF().show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------------+----------+--------+--------+--------+-----+----+--------------------+--------------+-------+--------+------+-------+
|centro|aniomes|fechaproceso|codcliente|codienda|cantidad|   monto|deuda|tipo|        process_code|     uuid_file|pt_year|pt_month|pt_day|pt_time|
+------+-------+------------+----------+--------+--------+--------+-----+----+--------------------+--------------+-------+--------+------+-------+
|  CR03| 202210|  2022-06-27| 200036215|     300|   27.00|47601.00|    0|   P|7d0eb092e92c4cc19...|20220709233628|   2022|      07|    09| 233628|
|  CR03| 202210|  2022-06-27| 200073557|     422|    2.00| 8053.11|    0|   P|7d0eb092e92c4cc19...|20220709233628|   2022|      07|    09| 233628|
+------+-------+------------+----------+--------+--------+--------+-----+----+--------------------+--------------+-------+--------+------+-------+
only showing top 2 rows

In [19]:
glue_raw = append_audit(glue_raw)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
glue_raw = append_partitions(glue_raw)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
glue_raw = data_sort_columns(glue_raw)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

glue_raw = Map.apply(frame = glue_raw, f = AddDateCol_year)
glue_raw = Map.apply(frame = glue_raw, f = AddDateCol_month)
glue_raw = Map.apply(frame = glue_raw, f = AddDateCol_day)
glue_raw = Map.apply(frame = glue_raw, f = AddDateCol_time)

In [24]:
glue_raw.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- centro: string
|-- aniomes: string
|-- fechaproceso: string
|-- codcliente: string
|-- codienda: string
|-- cantidad: string
|-- monto: string
|-- deuda: string
|-- tipo: string
|-- process_code: string
|-- uuid_file: string
|-- pt_year: string
|-- pt_month: string
|-- pt_day: string
|-- pt_time: string

In [25]:
glue_raw.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

33686

glue_raw.toDF().show(5)

glue_raw = glue_raw.select_fields(column_total)

glue_raw.toDF().show()

In [26]:
class Glueutils:
    
    def __init__(self, data_schema, data_interface, bucket_raw):
        self.glue = boto3.client('glue')
        self.data_schema = data_schema
        self.data_interface = data_interface
        self.database_key = "database"
        self.bucket_raw = bucket_raw
        
        self.GLUE_TABLE_FORMATS = {
            'csv': {
                'Input': 'org.apache.hadoop.mapred.TextInputFormat',
                'Output': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
                'Serde': {
                    'Lib': 'org.apache.hadoop.hive.serde2.OpenCSVSerde',
                    'Params': {
                        'separatorChar': ','
                    }
                },
                'Prefix': '',
                'Key': lambda prefix, table: f"{self.database_key}/{prefix}/{table}",
                'Bucket': self.bucket_raw
            },
            'tsv': {
                'Input': 'org.apache.hadoop.mapred.TextInputFormat',
                'Output': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
                'Serde': {
                    'Lib': 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe',
                    'Params': {
                        'field.delim': '\t'
                    }
                },
                'Prefix': '',
                'Key': lambda prefix, table: f"{self.database_key}/{prefix}/{table}",
                'Bucket': self.bucket_raw
            }
        }

        self.DEFAULT_PARTITION_KEYS = self.data_schema["partitions_key"]


    def table_spec(self, table_name: str, file_type: str, s3_path: str, columns: List[str], delimiter: Dict[str, dict]):
        """
        Returns a valid table spec for use with Glue CreateTable API
        """

        formats = self.GLUE_TABLE_FORMATS[file_type]

        return {
            'Name': table_name,
            'StorageDescriptor': {
                'Columns': columns,
                'Compressed': True,
                'Location': s3_path,
                'InputFormat': formats['Input'],
                'OutputFormat': formats['Output'],
                'SerdeInfo': {
                    'SerializationLibrary': formats['Serde']['Lib'],
                    'Parameters': formats['Serde']['Params']
                }
            },
            'PartitionKeys': self.DEFAULT_PARTITION_KEYS,
            'TableType': 'EXTERNAL_TABLE',
            'Parameters': {
                'EXTERNAL': 'TRUE',
                'classification': file_type,
                'creationDate': datetime.datetime.utcnow().isoformat()
            }
        }

    def table_spec_parquet(self, table_name: str, s3_path: str, columns: List[str]):
        """
        Returns a valid table spec for use with Glue CreateTable API
        """
        return {
            'Name': table_name,
            'StorageDescriptor': {
                'Columns': columns,
                'Compressed': True,
                'Location': s3_path,
                'InputFormat': 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat',
                'OutputFormat': 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat',
                'SerdeInfo': {
                    'SerializationLibrary': 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe',
                    'Parameters':{
                        'serialization.format': '1'
                    }
                },
            'Parameters': {
                'classification': 'parquet',
                'compressionType': 'none'
                }
            },
            'PartitionKeys': self.DEFAULT_PARTITION_KEYS,
            'TableType': 'EXTERNAL_TABLE',
            'Parameters': {
                'EXTERNAL': 'TRUE',
                'classification': 'parquet',
                'creationDate': datetime.datetime.utcnow().isoformat()
            }
        }
    
    def table_exists(self, database: str, name: str) -> bool:
        """
        Check if given table exists in the Glue catalog
        """
        resp = self.glue.get_tables(DatabaseName=database, Expression=f".*{name}.*")
        return len(resp['TableList']) > 0


    def create_table(self, system: str, interface: str, file_type: str, database_name: str):
        """
        Create table for given system and interface, if it exists
        Return name and warehouse path info for the table
        """
        source = self.data_interface["source"]
        interface = self.data_interface["interface"]
        table_name = self.data_schema["table_name"]
        table_name = f"{interface}_{table_name}"
        
        formats = self.GLUE_TABLE_FORMATS[file_type]
        # table_name = f"{formats['Prefix']}{system}_{interface}"
        bucket = formats['Bucket']
        key = f"{self.database_key}/{source}/{interface}/{table_name}"
        s3_path = f"s3://{bucket}/{key}"

        if self.table_exists(database_name, table_name):
            print('tabla ya existe')
            return table_name, bucket, key

        # prefix_schema = 'work/schemas'
        bucket_schema = bucket
        interface_schema = interface


        columns = self.data_schema["columns"]
        delimiter = self.data_schema["field.delim"]
        # revisar table_spec, para que se envie el formato correcto de parquet

        table = self.table_spec_parquet(table_name, s3_path, columns)

        try:
            self.glue.create_table(DatabaseName=database_name, TableInput=table)
        except self.glue.exceptions.AlreadyExistsException:
            print(f"{table_name} ya existe")

        return table_name, bucket, key


    def get_glue_table(self, database_name: str, table_name: str):
        try:
            response = self.glue.get_table(DatabaseName=database_name, Name=table_name)

            print(response)
            return response['Table']
        except glue.exceptions.EntityNotFoundException:
            raise ValueError(f'Error:   {database_name}.{table_name} no existe')
            return None


    def create_partition(self, database_name, table_name, new_partition):
        try:
            print('lm: agregar nueva particion: ', end='')
            response = self.glue.create_partition(DatabaseName=database_name,
                                             TableName=table_name,
                                             PartitionInput=new_partition)
            print('Okey particion agregada:')
        except glue.exceptions.AlreadyExistsException as e:
            print('lm:excepcion Particion ya existente', e)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
database_name_raw

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'auna-lake-raw-dev'

In [29]:
print('lm: creando tabla en Glue: ', end="")

GlueObj = Glueutils(data_schema, data_interface, bucket_raw)
table_name, table_bucket, table_key = GlueObj.create_table(system=data_interface["source"],
                                interface= data_interface["interface"],
                                file_type= data_schema["field.type"],
                                database_name= database_name_raw)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lm: creando tabla en Glue: test_tbl_faltante_virtual ya existe

In [30]:
table_key

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'database/sap/test/test_tbl_faltante_virtual'

In [31]:
raw_path_file = f"s3://{table_bucket}/{table_key}"
raw_path_file

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://auna-lake-raw-dev/database/sap/test/test_tbl_faltante_virtual'

In [32]:
partitions_key = [col["Name"] for col in data_schema["partitions_key"]]
partitions_key

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['pt_year', 'pt_month', 'pt_day', 'pt_time']

In [33]:
pt_year = data_interface['pt_year']
pt_month =data_interface['pt_month']
pt_day =data_interface['pt_day']
pt_time =data_interface['pt_time']

partition = f"pt_year={pt_year}/pt_month={pt_month}/pt_day={pt_day}/pt_time={pt_time}"

f"{raw_path_file}/{partition}"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://auna-lake-raw-dev/database/sap/test/test_tbl_faltante_virtual/pt_year=2022/pt_month=07/pt_day=09/pt_time=233628'

In [34]:

glueContext.purge_s3_path(f"{raw_path_file}/{partition}", options={"retentionPeriod":0})


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
glueContext.write_dynamic_frame.from_catalog(
        frame=glue_raw,
        database = database_name_raw, 
        table_name = table_name, 
        transformation_ctx = database_name_raw, 
        additional_options={"partitionKeys": partitions_key})

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

result =  glueContext.write_dynamic_frame \
    .from_options(
        frame = glue_raw, 
        connection_type = "s3", 
        connection_options =  {"path": raw_path_file, 
                               "partitionKeys": partitions_key},
        format = "parquet")

In [36]:
table = GlueObj.get_glue_table(database_name_raw, table_name)


print('lm: configurar nueva particion')
new_partition = {
    'Values': [pt_year, pt_month, pt_day, pt_time],
    'StorageDescriptor': {
        'InputFormat': 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat',
        'OutputFormat': 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat',
        'SerdeInfo': {
            'SerializationLibrary': 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe',
            'Parameters':{
                'serialization.format': '1'
            }
        },
        'Columns': data_schema["columns"],
        'Location': f"{raw_path_file}/{partition}"
    }
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'Table': {'Name': 'test_tbl_faltante_virtual', 'DatabaseName': 'auna-lake-raw-dev', 'CreateTime': datetime.datetime(2022, 7, 12, 2, 51, 6, tzinfo=tzlocal()), 'UpdateTime': datetime.datetime(2022, 7, 12, 2, 51, 6, tzinfo=tzlocal()), 'Retention': 0, 'StorageDescriptor': {'Columns': [{'Name': 'centro', 'Type': 'string'}, {'Name': 'aniomes', 'Type': 'string'}, {'Name': 'fechaproceso', 'Type': 'string'}, {'Name': 'codcliente', 'Type': 'string'}, {'Name': 'codienda', 'Type': 'string'}, {'Name': 'cantidad', 'Type': 'string'}, {'Name': 'monto', 'Type': 'string'}, {'Name': 'deuda', 'Type': 'string'}, {'Name': 'tipo', 'Type': 'string'}, {'Name': 'process_code', 'Type': 'string'}, {'Name': 'uuid_file', 'Type': 'string'}], 'Location': 's3://auna-lake-raw-dev/database/sap/test/test_tbl_faltante_virtual', 'InputFormat': 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat', 'OutputFormat': 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat', 'Compressed': True, 'NumberOfBucke

In [37]:
print('lm: agregar nueva particion: ', end='')
response = GlueObj.create_partition(database_name =database_name_raw,
                                      table_name= table_name,
                                      new_partition=new_partition)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lm: agregar nueva particion: lm: agregar nueva particion: Okey particion agregada: